In [48]:
import os
import re
import fitz
from tqdm import tqdm
from datasets import Dataset, DatasetDict

In [50]:
PDF_DIR_DIET = "./diet_pdf"
PDF_DIR_NUTRI = "./nutri_pdf"

TXT_DIR = "./domain_txt"
DATASET_DIR = "./domain_dataset"

os.makedirs(TXT_DIR, exist_ok=True)
os.makedirs(DATASET_DIR, exist_ok=True)

In [52]:
def pdf_to_txt(pdf_path: str) -> str:
    """
    Извлекает текст из PDF-файла, используя PyMuPDF (fitz).
    Возвращает сырой текст со всех страниц файла.
    """
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            blocks = page.get_text("blocks")
            for b in blocks:
                text += b[4] + "\n\n"
    return text

In [54]:
def clean_text(text: str) -> str:
    """
    Очищает входной текст от:
      - лишних переносов строк
      - URL-ссылок (http/https)
      - HTML-тегов (<...>)
      - ссылок-цитат вида [1], [2]
      - непечатаемых и управляющих символов
      - множественных пробелов
      - символов вне разрешённого набора
      - лишних пробелов перед знаками препинания
    Возвращает очищенный текст.
    """
    text = re.sub(r'\r\n', '\n', text)
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'\[\d+\]', '', text)
    text = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', text)
    text = re.sub(r'[ \t]+', ' ', text)
    text = re.sub(r'\s+([.,:;!?])', r'\1', text)
    return text.strip()

In [55]:
def process_pdf_folder(folder_path: str, label: str):
    pdf_files = [f for f in os.listdir(folder_path) if f.lower().endswith(".pdf")]

    texts = []
    total_chars_raw = 0
    total_chars_clean = 0

    for filename in tqdm(pdf_files, desc=f"Обработка PDF из {folder_path}"):
        pdf_path = os.path.join(folder_path, filename)
        txt_filename = os.path.splitext(filename)[0] + ".txt"
        txt_path = os.path.join(TXT_DIR, txt_filename)

        try:
            raw_text = pdf_to_txt(pdf_path)
            total_chars_raw += len(raw_text)

            cleaned = clean_text(raw_text)
            total_chars_clean += len(cleaned)

            texts.append({"text": cleaned, "label": label})

            with open(txt_path, "w", encoding="utf-8") as f:
                f.write(cleaned)

        except Exception as e:
            print(f"Ошибка при обработке файла {filename}: {e}")

    print(f"\n==== СТАТИСТИКА ДЛЯ ПАПКИ: {folder_path} ====")
    print(f"Всего PDF-файлов: {len(pdf_files)}")
    print(f"Общее кол-во символов ДО очистки: {total_chars_raw}")
    print(f"Общее кол-во символов ПОСЛЕ очистки: {total_chars_clean}")
    print(f"Общее кол-во элементов текста: {len(texts)}")
    print("==========================================\n")

    return texts

In [58]:
all_texts = []

In [59]:
diet_texts = process_pdf_folder(PDF_DIR_DIET, label="diet")
all_texts.extend(diet_texts)

Обработка PDF из ./diet_pdf: 100%|██████████| 28/28 [00:05<00:00,  5.40it/s]


==== СТАТИСТИКА ДЛЯ ПАПКИ: ./diet_pdf ====
Всего PDF-файлов: 28
Общее кол-во символов ДО очистки: 7813011
Общее кол-во символов ПОСЛЕ очистки: 7177470
Общее кол-во элементов текста: 28



In [61]:
nutri_texts = process_pdf_folder(PDF_DIR_NUTRI, label="nutri")
all_texts.extend(nutri_texts)

Обработка PDF из ./nutri_pdf: 100%|██████████| 6/6 [00:05<00:00,  1.18it/s]


==== СТАТИСТИКА ДЛЯ ПАПКИ: ./nutri_pdf ====
Всего PDF-файлов: 6
Общее кол-во символов ДО очистки: 6975218
Общее кол-во символов ПОСЛЕ очистки: 4710545
Общее кол-во элементов текста: 6



In [62]:
dataset = Dataset.from_list(all_texts)

split_dataset = dataset.train_test_split(test_size=0.05, seed=73)
ds = DatasetDict({
    "train": split_dataset["train"],
    "validation": split_dataset["test"]
})

ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 32
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2
    })
})

In [63]:
ds.save_to_disk(DATASET_DIR)

Saving the dataset (0/1 shards):   0%|          | 0/32 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2 [00:00<?, ? examples/s]